In [11]:
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime, timedelta
import random

In [12]:
# InfluxDB 설정
url = "http://localhost:8086"  
token = "TwrNN4J0ablMZFPFRB9GsUahs-uESVsM6WU0KKtY-jvJrFbhnxt7atnPg2wyU801B2GbFprJtomq5N1jog9uAg=="         
org = "gaonpf"             
bucket = "thermal_data"      

# 클라이언트 초기화
client = InfluxDBClient(url=url, token=token, org=org)

In [27]:
query_api = client.query_api()

# 최근 30초 동안의 데이터를 조회하는 쿼리
query = f'from(bucket:"{bucket}") |> range(start: -10m)'
tables = query_api.query(query, org=org)

for table in tables:
    for record in table.records:
        print(f"Time: {record.get_time()}, tagname: {record.values.get('tagName')}, Value: {record.get_value()}")


Time: 2024-08-07 04:45:41.600000+00:00, tagname: 1, Value: 37.09058815323103
Time: 2024-08-07 04:45:42.601000+00:00, tagname: 1, Value: 37.41194218207787
Time: 2024-08-07 04:45:43.603000+00:00, tagname: 1, Value: 36.51475271454793
Time: 2024-08-07 04:45:44.603000+00:00, tagname: 1, Value: 37.941140950198935
Time: 2024-08-07 04:45:45.604000+00:00, tagname: 1, Value: 36.50576382168554
Time: 2024-08-07 04:45:46.605000+00:00, tagname: 1, Value: 37.50948042534547
Time: 2024-08-07 04:45:47.606000+00:00, tagname: 1, Value: 36.58759226351328
Time: 2024-08-07 04:45:48.608000+00:00, tagname: 1, Value: 37.71744152639428
Time: 2024-08-07 04:45:49.609000+00:00, tagname: 1, Value: 37.13331689450047
Time: 2024-08-07 04:45:50.611000+00:00, tagname: 1, Value: 36.53405594110694
Time: 2024-08-07 04:45:51.612000+00:00, tagname: 1, Value: 37.45809691250566
Time: 2024-08-07 04:45:52.613000+00:00, tagname: 1, Value: 37.92396837505704
Time: 2024-08-07 04:45:53.614000+00:00, tagname: 1, Value: 37.3853161021038

In [9]:
import influxdb
import pandas as pd
from utils.singleton import SingletonInstance


class InfluxConnector(metaclass=SingletonInstance):
    def __init__(self, host: str, port: int, username: str, password: str, database: str) -> None:
        self.conn = influxdb.InfluxDBClient(host=host, port=port, username=username, password=password, database=database)

    def __create_query(self, tagnames: str | list[str], start: str, end: str):
        if isinstance(tagnames, str):
            tagnames = [tagnames]
        tag_conditions = " OR ".join([f"tagname = '{tagname}'" for tagname in tagnames])
        query = f"SELECT * FROM rawvalue WHERE ({tag_conditions}) and time >= '{start}' and time <= '{end}'"  # noqa: E501
        return query

    def __parse_influx_res(self, influx_res: influxdb.resultset.ResultSet) -> pd.DataFrame:
        series = influx_res.raw["series"]
        if not series:
            return pd.DataFrame()
        df = pd.DataFrame(series[0]["values"], columns=series[0]["columns"])
        return df

    def load_from_influx(self, tagnames: str | list[str], start: str, end: str) -> influxdb.resultset.ResultSet:
        query = self.__create_query(tagnames, start, end)
        res = self.conn.query(query)
        df = self.__parse_influx_res(res)
        return df

In [ ]:
InfluxConnector.load_from_influx()